# Baseline
## Import

In [2]:
# %pip install accelerate
# %pip install -i https://pypi.org/simple/ bitsandbytes
# %pip install transformers[torch] -U

# %pip install datasets
# %pip install langchain
# %pip install langchain_community
# %pip install PyMuPDF
# %pip install sentence-transformers
# %pip install faiss-gpu

In [3]:
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import os
import pickle

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Gemma2ForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from accelerate import Accelerator

# Train 관련
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, PeftConfig
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vector DB

In [4]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

## DB 생성

In [5]:
import os 

In [6]:
train_src_list = os.listdir(os.getcwd()+'/train_source')
print(len(train_src_list))
print(train_src_list)

16
['1-1 2024 주요 재정통계 1권.pdf', '2024 나라살림 예산개요.pdf', '2024년도 성과계획서(총괄편).pdf', '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf', '고용노동부_내일배움카드(일반).pdf', '고용노동부_조기재취업수당.pdf', '고용노동부_청년일자리창출지원.pdf', '국토교통부_민간임대(융자).pdf', '국토교통부_소규모주택정비사업.pdf', '국토교통부_전세임대(융자).pdf', '보건복지부_노인일자리 및 사회활동지원.pdf', '보건복지부_생계급여.pdf', '월간 나라재정 2023년 12월호.pdf', '재정통계해설.pdf', '중소벤처기업부_창업사업화지원.pdf']


,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10..."
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ..."
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ..."


In [7]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [6]:
test_df

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


## 모델 학습

In [8]:
# X_train = df['Question']
# Y_train = df['Answer']

# # make dataset
# train_data = []
# for x, y in zip(X_train, Y_train):
#     train_data.append({"user":x, "assistant":y})
    
# train_dataset = Dataset.from_list(train_data)

# dataset_dict = DatasetDict({
#     "train": train_dataset
# })

# # save at local
# dataset_dict.save_to_disk('./finance_dataset')

# load dataset from local
dataset_dict = DatasetDict.load_from_disk('./finance_dataset')

dataset_dict['train'][0]

{'user': '2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?',
 'assistant': '2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.'}

In [10]:
def load_model(model_id):
            # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 토크나이저 로드 및 설정
    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation='eager'
    )
    
    return model, tokenizer

def generate_prompts_train(example):
    output_texts = []
    for i in range(len(example['user'])):
        messages = [
#             {"role": "system", "content": system_msg},
            {'role': 'user', 'content': "{}".format(example['user'][i])},
            {'role': 'assistant', 'content': "{}".format(example['assistant'][i])}
        ]

        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        chat_message = chat_message.rstrip()
        chat_message = chat_message+"<eos>"
        
        '''
        print("###################################")
        print('###chat_message###\n', chat_message)
        print("###################################")
        '''

        output_texts.append(chat_message)

    return output_texts

def train_model_SFT(model_id, train_dataset, model_save_path):
    model, tokenizer = load_model(model_id)
    
    lora_config = LoraConfig(
        r=8,
        lora_alpha = 16,
        lora_dropout = 0.05,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
    )   

    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        max_seq_length=512,
        args=TrainingArguments(
            output_dir=model_save_path+'_checkpoint_dir',
#             evaluation_strategy = "steps",
#             eval_steps = 5,
#             load_best_model_at_end = True,
            num_train_epochs = 5,
            max_steps=3000,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            optim="paged_adamw_8bit",
            warmup_steps=100,
            learning_rate=2e-4,
            bf16=True, 
            logging_steps=100,
            report_to="wandb",
            run_name=model_save_path,
            push_to_hub=False
        ),
        peft_config=lora_config,
        formatting_func=generate_prompts_train
#         callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    trainer.train()
    
    # model save
    ADAPTER_MODEL = model_save_path+"lora_adapter"

    trainer.model.save_pretrained(ADAPTER_MODEL)

    model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.bfloat16)

    model = model.merge_and_unload()
    model.save_pretrained(model_save_path)
    
    return model, tokenizer

In [11]:
model, tokenizer = train_model_SFT(model_id='google/gemma-2-9b-it', 
                                   train_dataset=dataset_dict['train'], 
                                   model_save_path='finance_gemma2-9b-it_20240822_v2')

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.60s/it]
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: drtkdldjstm (drtkdldjstm-secudaim). Use `wandb login --relogin` to force relogin


Step,Training Loss
100,2.320600
200,1.081100
300,0.777200
400,0.542700
500,0.393100
600,0.291900
700,0.243500
800,0.214600
900,0.182200
1000,0.172700


/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


## Model Import

In [8]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID google/gemma-2-9b-it
    base_model = "google/gemma-2-9b-it"
    model_id = "finance_gemma2-9b-it_20240822_v2"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        #temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )
    
    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

# LLM 파이프라인
llm = setup_llm_pipeline()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.67s/it]
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and sho

In [10]:
# pdf_databases = process_pdfs_from_dataframe(test_df, base_directory)

# # Save the pdf_databases dictionary to a file
# with open('pdf_databases_test.pkl', 'wb') as f:
#     pickle.dump(pdf_databases, f)
#     print("pdf_databases has been saved to pdf_databases.pkl")

# Later, load the pdf_databases dictionary from the file
with open('pdf_databases_test.pkl', 'rb') as f:
    pdf_databases = pickle.load(f)
    print("pdf_databases has been loaded from pdf_databases.pkl")

pdf_databases has been loaded from pdf_databases.pkl


## Langchain을 이용한 추론

In [11]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:03<06:17,  3.90s/it]

Answer: 2,300,000백만원

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:17<15:39,  9.79s/it]

...

Answer: 국가결산보고서는 우발부채를 재무제표상 부채로 인식하지 않으며, 주석에 공시합니다. 지방자치단체 회계기준은 우발부채를 재무제표상 부채로 인식할 수 있습니다.

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [13:43<00:11, 11.46s/it]

Answer: 우발부채는 미래에 특정 사건이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 그 이상의 조건이 충족되어야 금융거래로 인식된다는 점에서 ‘부채’와 차이가 있다. 우발부채는 재정위험을 발생시키는 구체적 원인이 되며, 의도적인 공공정책이나 재정위기와 같이 예기치 못한 사건으로 인해 발생할 수 있다. 우발부채의 관리가 중요한 이유는 재정건전

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [13:55<00:00,  8.53s/it]

Answer: 보증은 피보증처의 부채에 대해 지급을 보증하는 것을 의미하며, 국가의 경우 국가채무에 준하는 관리가 필요한 보증채무로 관리된다. 표준화 보증은 은행의 대출을 보증하는 데 사용되며, 은행은 이를 통해 대출금리 인하 등의 경제적 편익을 얻을 수 있다.



## Submission

In [13]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./QLoRA_gemma2-9b-it_v2_submission.csv", encoding='UTF-8-sig', index=False)


In [14]:
submit_df

,SAMPLE_ID,Answer
0,TEST_000,"2,300,000백만원"
1,TEST_001,ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조"
3,TEST_003,재창업자금(실패 경영인에 대한 재기지원)
4,TEST_004,"미래기술육성자금, 고성장촉진자금"
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,현행법과 규정에 따라 이미 가입자가 획득한 미래 급여의 현재가치에서 현행법과 규정에...
95,TEST_095,"국가결산보고서는 우발부채를 재무제표상 부채로 인식하지 않으며, 주석에 공시합니다. ..."
96,TEST_096,"우발부채는 미래에 특정 사건이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 ..."


In [15]:
df = pd.read_csv("./QLoRA_gemma2-9b-it_v2_submission.csv")
df

,SAMPLE_ID,Answer
0,TEST_000,"2,300,000백만원"
1,TEST_001,ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조"
3,TEST_003,재창업자금(실패 경영인에 대한 재기지원)
4,TEST_004,"미래기술육성자금, 고성장촉진자금"
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,현행법과 규정에 따라 이미 가입자가 획득한 미래 급여의 현재가치에서 현행법과 규정에...
95,TEST_095,"국가결산보고서는 우발부채를 재무제표상 부채로 인식하지 않으며, 주석에 공시합니다. ..."
96,TEST_096,"우발부채는 미래에 특정 사건이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 ..."
